### Azure agent 

Coding quick start:

- pip install --pre azure-ai-projects>=2.0.0b1
- pip install azure-identity

In [71]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition
from dotenv import load_dotenv
import os 
load_dotenv()

True

In [72]:
# Set Foundry Project endpoint in the form
project_client = AIProjectClient(
    endpoint=os.environ.get("AZURE_EXISTING_AIPROJECT_ENDPOINT"),
    credential=DefaultAzureCredential(),
)

In [ ]:
premier_agent = next(iter(project_client.agents.list()), None)

if premier_agent is None:
    print("Aucun agent trouvé dans le projet.")
else:
    print(f"Agent(s) trouvé(s) dans le projet : \n Nom : {premier_agent.name} , \n Instruction : {premier_agent.versions.latest.definition.instructions} , \n Model : {premier_agent.versions.latest.definition.model}, \n Description : {premier_agent.versions.latest.description} ")

Agent(s) trouvé(s) dans le projet : 
 Nom : test-agent , 
 Instruction : You are a legal agent. Specialized in french law, answering only in French language with a hight relevance. , 
 Model : gpt-4.1-mini, 
 Description :  


In [ ]:
# Creates an agent, bumps the agent version if parameters have changed
agent = project_client.agents.create_version(  
    agent_name="test-agent",
    definition=PromptAgentDefinition(
            model=os.environ.get("MODEL_DEPLOYMENT_NAME"),
            instructions="You are a legal agent. Specialized in french law, answering only in French language with a hight relevance.",
            temperature=0,
            #Reasoning=,
            #tools=
        ),
)

In [ ]:
openai_client = project_client.get_openai_client()

In [ ]:

# Reference the agent to get a response
response = openai_client.responses.create(
    input=[{"role": "user", "content": "Tell me a one line story"}],
    extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
)

print(f"Response output: {response.output_text}")


